In [27]:
#create table players

import pymysql, csv, time
import datetime
with open('allPlayersLookup.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

conn = pymysql.connect(host='mysql.clarksonmsda.org', port=3306, user='ia626',
                       passwd='ia626clarkson', db='ia626', autocommit=True)
    
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("DROP TABLE IF EXISTS `geigersr_shots`;")

cur.execute("DROP TABLE IF EXISTS `geigersr_players`;")

sql = """CREATE TABLE `geigersr_players`
(
  `playerId` CHAR(7) NOT NULL,
  `name` VARCHAR(30) NOT NULL,
  `position` CHAR(1) NULL,
  `birthDate` DATE NULL,
  `weight` varchar(3) NULL,
  `height` VARCHAR(8) NULL,
  `nationality` CHAR(3) NULL,
  `shootsCatches` CHAR(1) NULL,
  `primaryNumber` varchar(2) NULL,
  `primaryPosition` CHAR(1) NULL,
  PRIMARY KEY (`playerId`)
);"""

cur.execute(sql)

sql = '''INSERT INTO `geigersr_players` (`playerId`,`name`,`position`,`birthDate`,`weight`,`height`,`nationality`,
  `shootsCatches`,`primaryNumber`,`primaryPosition`) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''

n = 0
blocksize = 500
tokens = []
for row in data:
    dto = None
    try:
        dto = datetime.datetime.strptime(row['birthDate'],"%m/%d/%Y")
    except Exception as e:
        #print(e)
        pass
    if dto is not None:
        dts = dto.strftime("%Y-%m-%d")
    else:
        dts = None
    tokens.append([row['playerId'],row['name'],row['position'],dts,
                    row['weight'],row['height'],row['nationality'],row['shootsCatches'],
                    row['primaryNumber'],row['primaryPosition']])

    if len(tokens) >= blocksize:
        cur.executemany(sql,tokens)
        tokens = []

    if n % 1000 == 0:
        print(n)

    n += 1

if len(tokens) > 0:
    cur.executemany(sql, tokens)

0
1000
2000
3000


In [1]:
import pymysql, csv, time
import datetime
with open('shots_2007-2022.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

In [2]:
#create table shots

import pymysql, csv, time
import datetime

conn = pymysql.connect(host='mysql.clarksonmsda.org', port=3306, user='ia626',
                       passwd='ia626clarkson', db='ia626', autocommit=True)
    
cur = conn.cursor(pymysql.cursors.DictCursor)

cur.execute("DROP TABLE IF EXISTS `geigersr_shots`;")

sql = ''' CREATE TABLE `geigersr_shots`
(
  `shotID` INT NOT NULL,
  `xCordAdjusted` INT NOT NULL,
  `yCordAdjusted` INT NOT NULL,
  `averageRestDifference` NUMERIC(7,2) NOT NULL,
  `event` VARCHAR(5) NOT NULL,
  `game_id` INT NOT NULL,
  `goal` INT NOT NULL,
  `isPlayoffGame` INT NOT NULL,
  `lastEventCategory` VARCHAR(10) NOT NULL,
  `season` INT NOT NULL,
  `shotAngle` NUMERIC(11,9) NOT NULL,
  `shotDistance` NUMERIC(12,9) NOT NULL,
  `shotGeneratedRebound` INT NOT NULL,
  `shotGoalieFroze` INT NOT NULL,
  `shotOnEmptyNet` INT NOT NULL,
  `shotRebound` INT NOT NULL,
  `shotRush` INT NOT NULL,
  `shotType` VARCHAR(10) NOT NULL,
  `shotWasOnGoal` INT NOT NULL,
  `team` CHAR(4) NOT NULL,
  `teamCode` CHAR(3) NOT NULL,
  `time` INT NOT NULL,
  `shooterId` char(7) NOT NULL,
  PRIMARY KEY (`shotid`),
  FOREIGN KEY (`shooterId`) REFERENCES `geigersr_players`(`playerId`)
);  '''

cur.execute(sql)

sql = '''INSERT INTO `geigersr_shots` ( `shotID`,`xCordAdjusted`,`yCordAdjusted`,`averageRestDifference`,`event`,
  `game_id`,`goal`,`isPlayoffGame`,`lastEventCategory`,`season`,`shotAngle`,`shotDistance`,`shotGeneratedRebound`,
  `shotGoalieFroze`,`shotOnEmptyNet`,`shotRebound`,`shotRush`,`shotType`,`shotWasOnGoal`,`team`,`teamCode`,`time`,`shooterId`) 
  VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'''


sql2 = '''SELECT `playerId` FROM `geigersr_players`'''

cur.execute(sql2)
l = []
for row in cur:
    l.append(row['playerId'])
#print(len(l))
noid = {}
n = 0
blocksize = 10000
tokens = []
for row in data:
    id = row['shooterPlayerId']
    id2 = id.split('.')
    id = id2[0]
    if id in l:
        tokens.append([row['shotID'],row['xCordAdjusted'],row['yCordAdjusted'],row['averageRestDifference'],row['event'],
                        row['game_id'],row['goal'],row['isPlayoffGame'],row['lastEventCategory'], row['season'], row['shotAngle'],
                        row['shotDistance'],row['shotGeneratedRebound'],row['shotGoalieFroze'], row['shotOnEmptyNet'],
                        row['shotRebound'],row['shotRush'],row['shotType'],row['shotWasOnGoal'],row['team'],row['teamCode'],
                        row['time'],id])

  
        if len(tokens) >= blocksize:
            cur.executemany(sql,tokens)
            tokens = []

        if n % 100000 == 0:
            print(n)
    else:
        if id in noid:
            noid[id] += 1
        else:
            noid[id] = 1
    n += 1
print(noid)
if len(tokens) > 0:
    cur.executemany(sql, tokens)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
{'': 39, '0': 57, '8466304': 111, '8470744': 4, '8457403': 87, '8458069': 155, '8467323': 9, '8467565': 35, '8462035': 117, '8455521': 129, '8458984': 92, '8473979': 123, '8467883': 56, '8458554': 98, '8458955': 186, '8462081': 238, '8467352': 74, '8466353': 101, '8458978': 91, '8447091': 153, '8466357': 36, '8458561': 73, '8473981': 55, '8468613': 54, '8467966': 139, '8473635': 42, '8462114': 130, '8465253': 60, '8456531': 49, '8462040': 64, '8457921': 52, '8458534': 96, '8467969': 132, '8470088': 40, '8467516': 118, '8452371': 93, '8465189': 86, '8469442': 39, '8459455': 129, '8467416': 184, '8458595': 123, '8467896': 57, '8458532': 232, '8467378': 13, '8457384': 45, '8448825': 58, '8468732': 3, '8467889': 16, '8469656': 4, '8469615': 2, '8470165': 47, '8459446': 39, '8458524': 83, '8462419': 45, '8462197': 90, '8468682': 1, '8466109': 26, '8468496': 90, '8

In [33]:
import pymysql, csv, time
import datetime
with open('shots_2007-2022.csv') as f:
    data = [{k: str(v) for k, v in row.items()}
        for row in csv.DictReader(f, skipinitialspace=True)]

conn = pymysql.connect(host='mysql.clarksonmsda.org', port=3306, user='ia626',
                       passwd='ia626clarkson', db='ia626', autocommit=True)
    



In [6]:
import math
cur = conn.cursor(pymysql.cursors.DictCursor)

sql = '''SELECT `playerId` FROM `geigersr_players`'''

cur.execute(sql)

l = []

for row in cur:
    l.append(row['playerId'])

print(len(l))
for row in data:
    id = row['shooterPlayerId']
    shooter = row['shooterName']
    #print(id)
    id2 = id.split('.')
    id = id2[0]
    if id in l:
        pass
    else:
        print(id)
    

3130
